In [ ]:
#setting up packages/etc for our code to run
!pip install geopandas
!pip install wbdata
!pip install cufflinks
!pip install iso3166

#import iso3166 #iso3166.countries.get('country details')
import wbdata
import cufflinks as cf
import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt
import seaborn as sns
cf.go_offline()
import geopandas


In [ ]:
# Start with an empty list of age-ranges and then construct the range we needed for future methods
#simplifies later methods
age_ranges = []
renames = {
    'Males 0004':'Males Ages 00-04',
    'Males 0509':'Males Ages 05-09',
    'Males 1014':'Males Ages 10-14',
    'Males 1519':'Males Ages 15-19',
    'Males 2024':'Males Ages 20-24',
    'Males 2529':'Males Ages 25-29',
    'Males 3034':'Males Ages 30-34',
    'Males 3539':'Males Ages 35-39',
    'Males 4044':'Males Ages 40-44',
    'Males 4549':'Males Ages 45-49',
    'Males 5054':'Males Ages 50-54',
    'Males 5559':'Males Ages 55-59',
    'Males 6064':'Males Ages 60-64',
    'Males 6569':'Males Ages 65-69',
    'Males 7074':'Males Ages 70-74',
    'Males 7579':'Males Ages 75-79',
    'Males 80UP':'Males Ages 80+',
    'Females 0004' : 'Females Ages 00-04',
    'Females 0509':'Females Ages 05-09',
    'Females 1014':'Females Ages 10-14',
    'Females 1519':'FemalesAges 15-19',
    'Females 2024':'Females Ages 20-24',
    'Females 2529':'Females Ages 25-29',
    'Females 3034':'Females Ages 30-34',
    'Females 3539':'Females Ages 35-39',
    'Females 4044':'Females Ages 40-44',
    'Females 4549':'Females Ages 45-49',
    'Females 5054':'Females Ages 50-54',
    'Females 5559':'Females Ages 55-59',
    'Females 6064':'Females Ages 60-64',
    'Females 6569':'Females Ages 65-69',
    'Females 7074':'Females Ages 70-74',
    'Females 7579':'Females Ages 75-79',
    'Females 80UP':'Females Ages 80+',
}
# Ranges top out at 80, and go in five year increments
for i in range(0,80,5):
    age_ranges.append(f"{i:02d}"+f"{i+4:02d}")

age_ranges.append("80UP")
male_variables = {"SP.POP."+age_range+".MA":"Males "+age_range for age_range in age_ranges}
female_variables = {"SP.POP."+age_range+".FE":"Females "+age_range for age_range in age_ranges}

In [ ]:
#Jesus B Soto:
# Give variable for clarity
#Here's an example of the populations we are looking at
variable_labels = {"SP.POP.TOTL":"Russia Population"}

world = wbdata.get_dataframe(variable_labels, country="RUS")

# Date index is of type string; change to integers
world.index = world.index.astype(int)

# Print a few years' data
world.head(30)

In [ ]:
#First part A: Population Statistics
def format_range(age_range):
  # Helper method for age formatting for API call to retrieve dataframe as illustrated in lecture
  low, high = age_range[0], age_range[1]
  if high > 80:
    formatted_range = [f"{i:02d}"+f"{i+4:02d}" for i in range(low, 80, 5)]
    formatted_range.append("80UP")
  else: 
    formatted_range = [f"{i:02d}"+f"{i+4:02d}" for i in range(low, high, 5)]
  return formatted_range

#included modulo operator in order to format function to be able to take age ranges outside of the increments of 5 years
def population(year, sex, age_range, place):
    start_increment = 0
    end_increment = 0
    age_range = list(age_range)
    if age_range[0] % 5 != 0:
        start_increment = age_range[0] % 5
        age_range[0] = age_range[0] - start_increment
    if age_range[1] % 5 != 0:
        end_increment = age_range[1] % 5
        age_range[1] = age_range[1] + (5 - age_range[1]%5)
    age_range = tuple(age_range)
    formatted_range = format_range(age_range)
    if sex == "Male":
        variables = {"SP.POP."+age_range+".MA":"Male"+age_range for age_range in formatted_range}
    elif sex == "Female":
        variables = {"SP.POP."+age_range+".FE":"Female"+age_range for age_range in formatted_range}
    df = wbdata.get_dataframe(variables, country=place)
    df = df.reset_index()
    df['date'] = df['date'].astype(int)
    result = df[df['date'] ==year]
    fixed_start = result.iloc[0][1:][0]*((5-start_increment)/5)
    fixed_end = (result.iloc[0][1:][len(result.iloc[0][1:])-1])*(end_increment+1)/5
    product = result.iloc[0][1:]
    product[0] = fixed_start
    product[len(product)-1] = fixed_end
    return sum(product)

In [ ]:
#unit test for code above:

assert population(year=2000,sex='Male',age_range=(0,100),place='WLD') > 3e9

In [ ]:
#Group A part 2
#they'll input region/country as area and year
#should take in an array of countries they want
#input years as [min,max]
def Groups_of_People(Area,years):
    renames = {
    'Males 0004':'Males Ages 00-04',
    'Males 0509':'Males Ages 05-09',
    'Males 1014':'Males Ages 10-14',
    'Males 1519':'Males Ages 15-19',
    'Males 2024':'Males Ages 20-24',
    'Males 2529':'Males Ages 25-29',
    'Males 3034':'Males Ages 30-34',
    'Males 3539':'Males Ages 35-39',
    'Males 4044':'Males Ages 40-44',
    'Males 4549':'Males Ages 45-49',
    'Males 5054':'Males Ages 50-54',
    'Males 5559':'Males Ages 55-59',
    'Males 6064':'Males Ages 60-64',
    'Males 6569':'Males Ages 65-69',
    'Males 7074':'Males Ages 70-74',
    'Males 7579':'Males Ages 75-79',
    'Males 80UP':'Males Ages 80+',
    'Females 0004' : 'Females Ages 00-04',
    'Females 0509':'Females Ages 05-09',
    'Females 1014':'Females Ages 10-14',
    'Females 1519':'FemalesAges 15-19',
    'Females 2024':'Females Ages 20-24',
    'Females 2529':'Females Ages 25-29',
    'Females 3034':'Females Ages 30-34',
    'Females 3539':'Females Ages 35-39',
    'Females 4044':'Females Ages 40-44',
    'Females 4549':'Females Ages 45-49',
    'Females 5054':'Females Ages 50-54',
    'Females 5559':'Females Ages 55-59',
    'Females 6064':'Females Ages 60-64',
    'Females 6569':'Females Ages 65-69',
    'Females 7074':'Females Ages 70-74',
    'Females 7579':'Females Ages 75-79',
    'Females 80UP':'Females Ages 80+',
}
    # Start with an empty list of age-rages
    age_ranges = []
    # Ranges top out at 80, and go in five year increments
    for i in range(0,80,5):
        age_ranges.append(f"{i:02d}"+f"{i+4:02d}")
        age_ranges.append("80UP")
        male_variables = {"SP.POP."+age_range+".MA":"Males "+age_range for age_range in age_ranges}
        female_variables = {"SP.POP."+age_range+".FE":"Females "+age_range for age_range in age_ranges}
    variables = male_variables
    variables.update(female_variables)
    df = wbdata.get_dataframe(variables,country=Area)
    df = df.rename(columns = renames)
    df = df.reset_index()
    df['date'] = df['date'].astype(int)
    df = df[df['date'] <= years[1]]
    df = df[df['date'] >= years[0]]
    if len(Area) > 1:
        df = df.set_index(['country','date'])
    else:#only one figure out 
        df['country'] = Area[0]
        df = df.set_index(['country','date'])
    return df

In [ ]:
#Creating a test for part b
test = Groups_of_People(['USA'],[2010,2019]).reset_index()
test = test.drop(columns = ['country'])
#test = test.set_index('date')
test.loc[0,:]

In [ ]:
#Part B (Population Pyramids)
#only takes a dataframe with just columns of age group and that's it
#must be indexed not by date or country (use default index, or reset index prior to use)
#Will need to manually reset the x axis labels to match the population of the country of interest
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
def pyramid(df):
    py.init_notebook_mode(connected=True)

    layout = go.Layout(barmode='overlay',
                       yaxis=go.layout.YAxis(range=[0, 90], title='Age'),
                       xaxis=go.layout.XAxis(title='Number'))


    bins = [go.Bar(x = df.filter(regex="Male").values,
                   y = [int(s[:2])+1 for s in age_ranges],
                   orientation='h',
                   name='Men',
                   marker=dict(color='blue'),
                   hoverinfo='skip'
                   ),

            go.Bar(x = -df.filter(regex="Female").values,
                   y=[int(s[:2])+1 for s in age_ranges],
                   orientation='h',
                   name='Women',
                   marker=dict(color='orange'),
                   hoverinfo='skip',
                   )
            ]
    fig = go.Figure(dict(data = bins, layout = layout))
    fig.update_xaxes(tickvals = [-10000000,-5000000,0,5000000,10000000])
    fig.update_xaxes(ticktext = ['10M','5M','0','5M','10M'])
    fig.show()

In [ ]:
#testing part B
pyramid(test.loc[0,:])

In [ ]:
#formatting our test dataframe to run 
test['date'] = test['date'].astype(str)
test = test.set_index('date')

In [ ]:
# C animated pyramid
#animated pyramid Part C Deliverable
#df fed into animated pyramid must be indexed by date and columns must be different age/sex ranges
#need to make the year range the range of years in the dataset manually
#the date column must be a string type, not an int or float
def animated_pyramid(df):
    def pyramid_builder(year):
        py.init_notebook_mode(connected=True)
        layout = go.Layout(barmode='overlay',
                   yaxis=go.layout.YAxis(range=[0, 90], title='Age'),
                   xaxis=go.layout.XAxis(title='Number'))
        bins = [go.Bar(x = df.loc[str(year),:].filter(regex="Male").values,
               y = [int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Men',
               marker=dict(color='blue'),
               hoverinfo='skip'
               ),
        go.Bar(x = -df.loc[str(year),:].filter(regex="Female").values,
               y=[int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Women',
               marker=dict(color='orange'),
               hoverinfo='skip',
               )
        ]
        fig = go.Figure(dict(data = bins, layout = layout))
        fig.update_xaxes(tickvals = [-10000000,-5000000,0,5000000,10000000])
        fig.update_xaxes(ticktext = ['10M','5M','0','5M','10M'])
        fig.show()
    ipywidgets.interact(pyramid_builder, year=(1961, 2018, 1))

In [ ]:
#importing package to make a slider
#year range must be within the range of our dataframe, set in the cell above
import ipywidgets
from ipywidgets import interactive, HBox, VBox
animated_pyramid(test)

In [ ]:
#Deliverable C (Population Maps) (Possibly Delete)
!pip install geopandas
!pip install wbdata
!pip install cufflinks
!pip install iso3166

import iso3166 #iso3166.countries.get('country details')
import wbdata
import cufflinks as cf
import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt
import seaborn as sns
cf.go_offline()
import geopandas

In [ ]:
#A function that draws a geopanda graph, accepting a query_dictionary, the year and the title as arguments
#countries_ussr = ['Armenia', 'Azerbaijan', 'Belarus', 'Estonia', 'Georgia', 'Kazakhstan', 'Kyrgyz Republic', 'Latvia', 'Lithuania', 'Moldova', 'Russia', 'Tajikistan', 'Turkmenistan', 'Ukraine', 'Uzbekistan']
#countries_iso3166 = ['ARM', 'AZE', 'BLR', 'EST', 'GEO','KAZ', 'KGZ', 'LVA', 'LTU', 'MDA', 'RUS', 'TJK', 'TKM', 'UKR', 'UZB' ]

#countries: Armenia', 'Azerbaijan', 'Belarus', 'Estonia', 'Georgia', 'Kazakhstan', 'Kyrgyz Republic', 'Latvia', 'Lithuania', 'Moldova', 'Russia', 'Tajikistan', 'Turkmenistan', 'Ukraine', 'Uzbekistan'

def geopand_ussr(indi_dict, year, title):
    countries_ussr = ['Armenia', 'Azerbaijan', 'Belarus', 'Estonia', 'Georgia','Kyrgyzstan', 'Kazakhstan', 'Latvia', 'Lithuania', 'Moldova', 'Russia','Tajikistan', 'Turkmenistan', 'Ukraine', 'Uzbekistan']
    countries_iso3166 = ['ARM', 'AZE', 'BLR', 'EST', 'GEO','KAZ', 'KGZ', 'LVA', 'LTU', 'MDA', 'RUS','TJK', 'TKM', 'UKR', 'UZB']
    
    df = wbdata.get_dataframe(indicators=indi_dict, country=countries_iso3166)
    df.reset_index(inplace=True)
    df['date'] = df['date'].astype(int)
    df = df[df['date'] == year]
    df = df.sort_values(by=['country'])

    world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
    world = world[world['name'].isin(countries_ussr)]
    world = world.sort_values(by=['name'])
    world['data_column'] = df['data_column'].values

    graph = world.plot('data_column', cmap='Spectral', figsize=(20,20), legend=True, legend_kwds={'shrink': 0.75}, aspect= 'auto', edgecolor= 'black')
    
    graph.set_xlim(right = 185)
    graph.set_xlim(left = -1)
    
    graph.set_title(title, size=22, pad=20);

    world.apply(lambda x: graph.annotate(text=x['name'], xy=x.geometry.centroid.coords[0], 
                                         ha='center'),axis=1);
    


In [ ]:
#indicator for year 1991: GDP per capita growth (annual %)
#note that the Baltic countries did not have data between period 1991-1994
indi_dict = {'NY.GDP.PCAP.KD.ZG': 'data_column'}
geopand_ussr(indi_dict, 1991, 'GDP per capita growth 1991 (%)')

In [ ]:
#agricultural supply list
!pip install eep153_tools
!pip install python_gnupg
#from eep153_tools.sheets import decrypt_credentials
#decrypt_credentials('../students.json.gpg')

In [ ]:
!ls ~/.eep153.service_accounts/

In [ ]:
from eep153_tools.sheets import read_sheets

#### Read a bunch of google worksheets into a dictionary of dataframes
data = read_sheets('https://docs.google.com/spreadsheets/d/1DLn9owcS7ggojJGWlI9vKSz0hqozn6cbcqNGWgzMZ8k',force_numeric=False)

data.keys()

In [ ]:
#graphing all the different countries growth rates
import numpy as np

variable_labels = {"SP.POP.TOTL":"Population"}

# Three letter codes come from wbdata.get_country()
countries = {"WLD":"World",
             "AZE":"Azerbaijan",
             "BLR":"Belarus",
             "EST":"Estonia",
             "GEO":"Georgia",
             "KAZ":"Kazakhstan",
             'KGZ':"Kyrgyz",
             "LVA":'Latvia',
             "LTU":"Lithuania",
             'MDA':'Moldova',
             'RUS':'Russia',
             "TJK":'Tajikistan',
             "TKM":"Turkmenistan",
             'UKR':'Ukraine',
             'UZB':'Uzbekistan',
             'USA':'United State',
             'CHN':'China',
             
            }

df = wbdata.get_dataframe(variable_labels, country = countries).squeeze()

df = df.unstack('country')
# Date index is of type string; change to integers
df.index = df.index.astype(int)

# Differences (over time) in logs give us growth rates
np.log(df).diff().iplot(title="Population Growth Rate in Diff Countries",
                        yTitle="Growth Rate",xTitle='Year')

In [ ]:
#countries like Uzbekistan, Turkmenistan, russia federation, azerbaijan, Ukraine, Moldova, Lithuania
#Estonia, Georgia, Kazakhstan, Kyrgyz Republic, Latvia, Tajikstan, turkmenistan ~2% drop
#Georgia had a massive 5% drop
#Kyrgyz republic had a drop but then a qucik then steady recovery, same with Kazakhstan
#countries ending 2020 upward: uzbekistan, Lithuania, Latvia, Kyrgyz Repub, Estonia
#countried ending 2020 downward: ukraine, turkmenistan, tajikistan, russia federation,MOLDOVA, Belarus,AzerBaijan
#examine why Georgia got wrecked are there any big differences?

In [ ]:
#lets examine why Georgia's so bad
#gives us all agricultural values of soviet union countries
#filtered to ~5 years before soviet union collapse to the present
import pandas as pd

def get_international_ag_productivity_data(series):

    columns=series.iloc[1,:]  # Need to fix columns
    series = series.iloc[2:,:]
    series.columns = columns

    # Set index to country code
    series = series.set_index('WDI Code')

    series = series[['%d' % t for t in range(1961,2020)]]

    series = series.replace({',':''},regex=True) # Get rid of commas in number strings
    series = series.replace({'':'NaN'}) # Change empty cells to NaN strings
    series = series.apply(lambda x: pd.to_numeric(x,errors='ignore'))
    series = series.astype(float) # Convert to floats

    series = series.stack()
    
    series = series.loc[~series.index.duplicated(keep='first')] 

    series.index.names = ['WDI Code','Year']

    return series
   

Data = ['Output','Ag TFP','Ag Land','Irrig','Pasture','Labor','Livestock',
        'Machinery','Fertilizer','Feed']

D = {}
for key in Data:
    D[key] = get_international_ag_productivity_data(data[key])

ag = pd.DataFrame(D)
ag = ag.reset_index()
ag = ag[ag['WDI Code'].isin(['ARM','AZE','BLR','EST','GEO','KAZ','KGZ','LVA','LTU','MDA','RUS','TJK','TKM','UKR','UZB'])]
ag = ag[ag['Year'].astype(int) > 1985]
ag = ag.set_index('WDI Code')
#ag[ag['Year'].astype(int) == 2016]
ag

In [ ]:
ag = ag.reset_index()

In [ ]:
#graphs of all of the different agricultural variables over time
for variable in ['Output','Ag TFP','Ag Land','Irrig','Pasture','Labor','Livestock','Machinery','Fertilizer','Feed']:
    test = ag[['Year',variable,'WDI Code']]
    test['Year'] = test['Year'].astype(int)
    fig, ax = plt.subplots(figsize=(18,6))
    for name, group in test.groupby('WDI Code'):
        group.plot(x= 'Year', y=variable, ax=ax, label=name,title = variable)
    plt.axvline(x = 1991, color = 'red', linestyle = '--')
    plt.show()

In [ ]:
#compare world average excluding countries vs our countries later